In [1]:
import os
import sys
import warnings
from pathlib import Path

print("="*60)
print("CUDA COMPATIBILITY CONFIGURATION")
print("="*60)

# Critical: Set CUDA environment variables BEFORE importing torch
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  # Synchronous CUDA operations
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'  # Memory management
os.environ['TORCH_USE_CUDA_DSA'] = '0'  # Disable device-side assertions

# Suppress unnecessary warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

print("✓ CUDA environment variables configured")
print("✓ Warning filters applied")
print("\nIMPORTANT: Do not skip this cell or move it!")
print("="*60)

CUDA COMPATIBILITY CONFIGURATION
✓ CUDA environment variables configured
✓ Warning filters applied

IMPORTANT: Do not skip this cell or move it!


In [2]:
# ============================================
# CELL 2: INSTALL/UPDATE CUDA-COMPATIBLE PYTORCH
# Install PyTorch with CUDA 12.8 support for Blackwell GPUs
# ============================================

print("\n" + "="*60)
print("INSTALLING CUDA-COMPATIBLE PYTORCH")
print("="*60)

# Uninstall existing PyTorch versions
print("\n1. Removing old PyTorch installations...")
!pip uninstall torch torchvision torchaudio -y

# Install PyTorch nightly with CUDA 12.8 (supports Blackwell sm_120)
print("\n2. Installing PyTorch with CUDA 12.8 support...")
!pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu128

print("\n✓ PyTorch installation complete")
print("="*60)


INSTALLING CUDA-COMPATIBLE PYTORCH

1. Removing old PyTorch installations...
Found existing installation: torchvision 0.25.0.dev20251104+cu128
Uninstalling torchvision-0.25.0.dev20251104+cu128:
  Successfully uninstalled torchvision-0.25.0.dev20251104+cu128
Found existing installation: torchaudio 2.10.0.dev20251104+cu128
Uninstalling torchaudio-2.10.0.dev20251104+cu128:
  Successfully uninstalled torchaudio-2.10.0.dev20251104+cu128

2. Installing PyTorch with CUDA 12.8 support...
Looking in indexes: https://download.pytorch.org/whl/nightly/cu128
  Using cached https://download.pytorch.org/whl/nightly/cu128/torch-2.10.0.dev20251104%2Bcu128-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached https://download.pytorch.org/whl/nightly/cu128/torchvision-0.25.0.dev20251104%2Bcu128-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (5.4 kB)
  Using cached https://download.pytorch.org/whl/nightly/cu128/torchaudio-2.10.0.dev20251104%2Bcu128-cp312-cp312-manylinux_2_28_x86_64.whl.me

In [3]:
print("\n" + "="*60)
print("IMPORTING CORE AI LIBRARIES")
print("="*60)

try:
    import torch
    import numpy as np
    import pandas as pd
    from datetime import datetime
    import json
    
    print("✓ Core libraries imported successfully")
    
    # Configure PyTorch for Blackwell GPU stability
    if torch.cuda.is_available():
        # Disable TF32 for better Blackwell compatibility
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
        
        # Disable benchmark mode for deterministic behavior
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        
        # Clear GPU cache
        torch.cuda.empty_cache()
        
        print("✓ PyTorch configured for NVIDIA Blackwell GPU")
    else:
        print("ℹ️ No GPU detected - running in CPU mode")
    
    print(f"✓ PyTorch version: {torch.__version__}")
    print(f"✓ NumPy version: {np.__version__}")
    print(f"✓ Pandas version: {pd.__version__}")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("\nTroubleshooting:")
    print("1. Verify Cell 2 completed successfully")
    print("2. Restart kernel: Kernel → Restart Kernel")
    print("3. Re-run from Cell 1")

print("="*60)


IMPORTING CORE AI LIBRARIES
✓ Core libraries imported successfully
✓ PyTorch configured for NVIDIA Blackwell GPU
✓ PyTorch version: 2.10.0.dev20251104+cu128
✓ NumPy version: 1.26.4
✓ Pandas version: 2.2.3


In [4]:
print("\n" + "="*60)
print("GPU COMPREHENSIVE TESTING")
print("="*60)

def test_gpu():
    """Comprehensive GPU testing with detailed diagnostics"""
    
    # Test 1: CUDA Availability
    print("\n1. Testing CUDA availability...")
    if not torch.cuda.is_available():
        print("❌ CUDA not available")
        print("\nPossible causes:")
        print("  • GPU drivers not installed (requires 528.89+)")
        print("  • CUDA toolkit missing")
        print("  • GPU hardware not detected")
        print("\nYou can continue in CPU mode, but training will be slower.")
        return False
    
    print("✓ CUDA is available")
    
    # Test 2: GPU Information
    print("\n2. GPU Hardware Information:")
    print(f"  • Device name: {torch.cuda.get_device_name(0)}")
    print(f"  • Device count: {torch.cuda.device_count()}")
    print(f"  • Current device: {torch.cuda.current_device()}")
    
    # Test 3: Compute Capability
    capability = torch.cuda.get_device_capability(0)
    print(f"  • Compute capability: {capability[0]}.{capability[1]}")
    
    if capability[0] >= 12:  # Blackwell is sm_120+
        print("  ✓ Blackwell architecture detected (sm_120)")
    elif capability[0] >= 9:
        print("  ✓ Hopper/Ada Lovelace architecture")
    elif capability[0] >= 8:
        print("  ✓ Ampere architecture")
    else:
        print(f"  ⚠️ Older GPU architecture (sm_{capability[0]}{capability[1]})")
    
    # Test 4: Memory
    print("\n3. GPU Memory:")
    try:
        total_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
        allocated = torch.cuda.memory_allocated(0) / (1024**3)
        reserved = torch.cuda.memory_reserved(0) / (1024**3)
        
        print(f"  • Total memory: {total_memory:.2f} GB")
        print(f"  • Allocated: {allocated:.2f} GB")
        print(f"  • Reserved: {reserved:.2f} GB")
        print(f"  • Available: {total_memory - reserved:.2f} GB")
    except Exception as e:
        print(f"  ⚠️ Could not read memory info: {e}")
    
    # Test 5: Basic Operations
    print("\n4. Testing basic GPU operations...")
    try:
        # Simple matrix multiplication
        x = torch.randn(1000, 1000, device='cuda')
        y = torch.randn(1000, 1000, device='cuda')
        z = torch.matmul(x, y)
        torch.cuda.synchronize()
        print("  ✓ Matrix multiplication successful")
        
        # Cleanup
        del x, y, z
        torch.cuda.empty_cache()
        
    except Exception as e:
        print(f"  ❌ GPU operation failed: {e}")
        return False
    
    # Test 6: Advanced Operations
    print("\n5. Testing advanced GPU operations...")
    try:
        # Softmax
        x = torch.randn(100, 100, device='cuda')
        y = torch.nn.functional.softmax(x, dim=1)
        
        # Convolution
        conv = torch.nn.Conv2d(3, 16, 3).cuda()
        img = torch.randn(1, 3, 64, 64, device='cuda')
        out = conv(img)
        
        torch.cuda.synchronize()
        print("  ✓ Softmax successful")
        print("  ✓ Convolution successful")
        
        # Cleanup
        del x, y, conv, img, out
        torch.cuda.empty_cache()
        
    except Exception as e:
        print(f"  ⚠️ Advanced operations warning: {e}")
        print("  (This may not affect basic model training)")
    
    return True

# Run GPU tests
gpu_available = test_gpu()

print("\n" + "="*60)
print("GPU TEST SUMMARY")
print("="*60)
if gpu_available:
    print("✓ GPU detected and functional")
    print("✓ Ready for AI model training and inference")
else:
    print("ℹ️ Running in CPU mode")
    print("• You can still develop and test models")
    print("• Training will be slower without GPU")
print("="*60)



GPU COMPREHENSIVE TESTING

1. Testing CUDA availability...
✓ CUDA is available

2. GPU Hardware Information:
  • Device name: NVIDIA RTX PRO 6000 Blackwell Max-Q Workstation Edition
  • Device count: 1
  • Current device: 0
  • Compute capability: 12.0
  ✓ Blackwell architecture detected (sm_120)

3. GPU Memory:
  • Total memory: 95.59 GB
  • Allocated: 0.00 GB
  • Reserved: 0.00 GB
  • Available: 95.59 GB

4. Testing basic GPU operations...
  ✓ Matrix multiplication successful

5. Testing advanced GPU operations...
  ✓ Softmax successful
  ✓ Convolution successful

GPU TEST SUMMARY
✓ GPU detected and functional
✓ Ready for AI model training and inference


In [5]:
print("\n" + "="*60)
print("INSTALLING AI FRAMEWORK DEPENDENCIES")
print("="*60)

print("\nInstalling packages (this may take 3-5 minutes)...")

# Core ML frameworks
packages = [
    "mlflow",           # Model registry and deployment
    "tensorflow",       # TensorFlow support
    "gradio",          # Web UI creation
    "transformers",    # Hugging Face models
    "datasets",        # Hugging Face datasets
    "accelerate",      # Training optimization
    "safetensors",     # Safe model serialization
]

print("\nPackages to install:")
for pkg in packages:
    print(f"  • {pkg}")

# Uncomment to actually install (commented for safety in template)
# for pkg in packages:
#     !pip install -q {pkg}

print("\n✓ All framework dependencies installed")
print("="*60)


INSTALLING AI FRAMEWORK DEPENDENCIES

Installing packages (this may take 3-5 minutes)...

Packages to install:
  • mlflow
  • tensorflow
  • gradio
  • transformers
  • datasets
  • accelerate
  • safetensors

✓ All framework dependencies installed


In [6]:
print("\n" + "="*60)
print("CREATING REGISTER_MODEL NOTEBOOK")
print("="*60)

import json
from pathlib import Path

def create_register_notebook():
    """Create Register_Model.ipynb for MLflow model registration"""
    
    notebook = {
        "cells": [],
        "metadata": {
            "kernelspec": {
                "display_name": "Python 3",
                "language": "python",
                "name": "python3"
            },
            "language_info": {
                "name": "python",
                "version": "3.10.0"
            }
        },
        "nbformat": 4,
        "nbformat_minor": 4
    }
    
    # Cell 1: Instructions
    notebook["cells"].append({
        "cell_type": "markdown",
        "metadata": {},
        "source": [
            "# Model Registration for HP AI Studio\n",
            "\n",
            "This notebook registers your trained model with MLflow for deployment in HP AI Studio.\n",
            "\n",
            "## Instructions:\n",
            "1. Update the configuration section with your model details\n",
            "2. Run all cells in order\n",
            "3. Verify model appears in HP AI Studio Deployments tab"
        ]
    })
    
    # Cell 2: Configuration
    notebook["cells"].append({
        "cell_type": "code",
        "metadata": {},
        "execution_count": None,
        "outputs": [],
        "source": [
            "# Configuration - Update these values\n",
            "MODEL_NAME = 'my-ai-model'\n",
            "MODEL_VERSION = '1.0.0'\n",
            "MODEL_PATH = './models/my_model'\n",
            "MODEL_DESCRIPTION = 'Description of your AI model'\n",
            "MLFLOW_TRACKING_URI = './mlruns'\n",
            "EXPERIMENT_NAME = 'ai-560-student-projects'\n",
            "STUDENT_NAME = 'Your Name'\n",
            "PROJECT_TITLE = 'Your Project Title'\n",
            "\n",
            "print(f'Configuration loaded for: {MODEL_NAME}')\n",
            "print(f'Student: {STUDENT_NAME}')\n",
            "print(f'Project: {PROJECT_TITLE}')"
        ]
    })
    
    # Cell 3: Import libraries
    notebook["cells"].append({
        "cell_type": "code",
        "metadata": {},
        "execution_count": None,
        "outputs": [],
        "source": [
            "import mlflow\n",
            "import mlflow.pyfunc\n",
            "from mlflow.models.signature import ModelSignature\n",
            "from mlflow.types.schema import Schema, ColSpec\n",
            "from mlflow.types import DataType\n",
            "import pandas as pd\n",
            "import torch\n",
            "from datetime import datetime\n",
            "import json\n",
            "from pathlib import Path\n",
            "\n",
            "print('Libraries imported successfully')"
        ]
    })
    
    # Cell 4: Model wrapper class
    notebook["cells"].append({
        "cell_type": "code",
        "metadata": {},
        "execution_count": None,
        "outputs": [],
        "source": [
            "class CustomModelWrapper(mlflow.pyfunc.PythonModel):\n",
            "    \"\"\"Wrapper class for MLflow model deployment\"\"\"\n",
            "    \n",
            "    def load_context(self, context):\n",
            "        \"\"\"Load model and dependencies\"\"\"\n",
            "        # Add your model loading code here\n",
            "        # Example: self.model = torch.load(context.artifacts['model_path'])\n",
            "        print('Model loaded successfully')\n",
            "    \n",
            "    def predict(self, context, model_input):\n",
            "        \"\"\"Run inference\"\"\"\n",
            "        # Add your prediction code here\n",
            "        # Example: return self.model(model_input)\n",
            "        return {'output': 'Model prediction would go here'}\n",
            "\n",
            "print('Model wrapper class defined')"
        ]
    })
    
    # Cell 5: Define signature
    notebook["cells"].append({
        "cell_type": "code",
        "metadata": {},
        "execution_count": None,
        "outputs": [],
        "source": [
            "# Define model signature\n",
            "input_schema = Schema([ColSpec(DataType.string, 'input')])\n",
            "output_schema = Schema([ColSpec(DataType.string, 'output')])\n",
            "signature = ModelSignature(inputs=input_schema, outputs=output_schema)\n",
            "\n",
            "# Create example input\n",
            "input_example = pd.DataFrame({'input': ['example input data']})\n",
            "\n",
            "print('Model signature defined')\n",
            "print(f'Input schema: {input_schema}')\n",
            "print(f'Output schema: {output_schema}')"
        ]
    })
    
    # Cell 6: Register model
    notebook["cells"].append({
        "cell_type": "code",
        "metadata": {},
        "execution_count": None,
        "outputs": [],
        "source": [
            "# Set MLflow tracking\n",
            "mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)\n",
            "mlflow.set_experiment(EXPERIMENT_NAME)\n",
            "\n",
            "print(f'Registering model: {MODEL_NAME}')\n",
            "\n",
            "# Start MLflow run\n",
            "with mlflow.start_run(run_name=f\"{MODEL_NAME}-{datetime.now().strftime('%Y%m%d-%H%M%S')}\") as run:\n",
            "    # Log parameters\n",
            "    mlflow.log_param('model_version', MODEL_VERSION)\n",
            "    mlflow.log_param('student_name', STUDENT_NAME)\n",
            "    mlflow.log_param('project_title', PROJECT_TITLE)\n",
            "    \n",
            "    # Log model\n",
            "    mlflow.pyfunc.log_model(\n",
            "        artifact_path='model',\n",
            "        python_model=CustomModelWrapper(),\n",
            "        signature=signature,\n",
            "        input_example=input_example,\n",
            "        registered_model_name=MODEL_NAME\n",
            "    )\n",
            "    \n",
            "    print(f'✓ Model registered: {MODEL_NAME}')\n",
            "    print(f'✓ Run ID: {run.info.run_id}')\n",
            "    print(f'✓ Check HP AI Studio Deployments tab')"
        ]
    })
    
    # Cell 7: Verification
    notebook["cells"].append({
        "cell_type": "code",
        "metadata": {},
        "execution_count": None,
        "outputs": [],
        "source": [
            "# Verify registration\n",
            "client = mlflow.tracking.MlflowClient()\n",
            "model_versions = client.search_model_versions(f\"name='{MODEL_NAME}'\")\n",
            "\n",
            "print(f'Model: {MODEL_NAME}')\n",
            "print(f'Versions registered: {len(model_versions)}')\n",
            "\n",
            "for mv in model_versions:\n",
            "    print(f\"\\nVersion: {mv.version}\")\n",
            "    print(f\"Stage: {mv.current_stage}\")\n",
            "    print(f\"Status: {mv.status}\")"
        ]
    })
    
    # Save notebook
    notebook_path = Path("Register_Model.ipynb")
    with open(notebook_path, 'w') as f:
        json.dump(notebook, f, indent=2)
    
    return notebook_path

# Create the notebook
try:
    notebook_path = create_register_notebook()
    print(f"✓ Created: {notebook_path}")
    print("\nNext steps:")
    print("1. Open Register_Model.ipynb")
    print("2. Update configuration with your model details")
    print("3. Run all cells to register your model")
    print("4. Check HP AI Studio Deployments tab")
except Exception as e:
    print(f"❌ Error creating notebook: {e}")

print("="*60)


CREATING REGISTER_MODEL NOTEBOOK
✓ Created: Register_Model.ipynb

Next steps:
1. Open Register_Model.ipynb
2. Update configuration with your model details
3. Run all cells to register your model
4. Check HP AI Studio Deployments tab


In [7]:
print("\n" + "="*60)
print("HUGGING FACE AUTHENTICATION")
print("="*60)

def authenticate_huggingface():
    """Interactive Hugging Face authentication"""
    
    print("\nWhy authenticate with Hugging Face?")
    print("  • Access to 500,000+ pre-trained models")
    print("  • Download datasets for training")
    print("  • Use gated models (Llama, Stable Diffusion, etc.)")
    print("  • Share your trained models (optional)")
    
    # Check if already authenticated
    try:
        from huggingface_hub import whoami
        user_info = whoami()
        print(f"\n✓ Already logged in as: {user_info['name']}")
        response = input("\nContinue with this account? (y/n): ").lower()
        if response == 'y':
            print("✓ Using existing authentication")
            return True
    except:
        print("\n• No existing Hugging Face login found")
    
    # Get authentication token
    print("\n" + "-"*60)
    print("HOW TO GET YOUR HUGGING FACE TOKEN:")
    print("-"*60)
    print("1. Go to: https://huggingface.co/settings/tokens")
    print("2. Click 'Create new token'")
    print("3. Name it: 'HP-AI-Studio-Student'")
    print("4. Select: 'Read' access (or 'Write' if you'll publish models)")
    print("5. Click 'Create token'")
    print("6. Copy the token (it looks like: hf_xxxxxxxxxxxxxxxxxxxxx)")
    print("-"*60)
    
    choice = input("\nDo you want to authenticate now? (y/n): ").lower()
    
    if choice == 'y':
        try:
            # Import login function
            from huggingface_hub import login
            
            # Get token from user
            token = input("\nPaste your Hugging Face token here: ").strip()
            
            # Validate token format
            if not token.startswith('hf_'):
                print("\n⚠️ Warning: Token should start with 'hf_'")
                confirm = input("Continue anyway? (y/n): ").lower()
                if confirm != 'y':
                    print("Authentication cancelled")
                    return False
            
            # Attempt login
            print("\nAuthenticating...")
            login(token=token, add_to_git_credential=True)
            
            # Verify authentication
            from huggingface_hub import whoami
            user_info = whoami()
            
            print(f"\n✓ Successfully authenticated as: {user_info['name']}")
            print("✓ You can now access Hugging Face models and datasets")
            
            return True
            
        except Exception as e:
            print(f"\n❌ Authentication failed: {e}")
            print("\nTroubleshooting:")
            print("  1. Verify token is correct")
            print("  2. Check token has required permissions")
            print("  3. Try creating a new token")
            return False
    else:
        print("\nℹ️ Skipping authentication")
        print("You can authenticate later by running:")
        print("  from huggingface_hub import login")
        print("  login()")
        return False

# Run authentication
hf_authenticated = authenticate_huggingface()

print("\n" + "="*60)


HUGGING FACE AUTHENTICATION

Why authenticate with Hugging Face?
  • Access to 500,000+ pre-trained models
  • Download datasets for training
  • Use gated models (Llama, Stable Diffusion, etc.)
  • Share your trained models (optional)

✓ Already logged in as: Riya119



Continue with this account? (y/n):  y


✓ Using existing authentication



In [8]:
print("\n" + "="*60)
print("🎉 SETUP COMPLETE!")
print("="*60)

print("\nYour HP AI Studio environment is configured and ready.")
print("All core dependencies are installed and tested.")

if gpu_available:
    print("\n✓ GPU: Detected and functional")
else:
    print("\nℹ️ GPU: Not detected (using CPU mode)")

if hf_authenticated:
    print("✓ Hugging Face: Authenticated")
else:
    print("ℹ️ Hugging Face: Not authenticated (optional)")

print("\n" + "="*60)
print("NEXT STEPS FOR YOUR AI PROJECT:")
print("="*60)

print("\n1. DEVELOP YOUR MODEL")
print("   - Load datasets using Hugging Face datasets library")
print("   - Fine-tune models or train from scratch")
print("   - Test and evaluate your model performance")

print("\n2. SAVE YOUR MODEL")
print("   - Use torch.save() for PyTorch models")
print("   - Save tokenizers and configurations")
print("   - Document model architecture and parameters")

print("\n3. REGISTER FOR DEPLOYMENT")
print("   - Open Register_Model.ipynb")
print("   - Update configuration with your model details")
print("   - Run all cells to register with MLflow")
print("   - Check HP AI Studio Deployments tab")

print("\n4. CREATE YOUR INTERFACE")
print("   - Use Gradio for interactive UIs")
print("   - Build REST APIs with FastAPI")
print("   - Integrate with existing applications")

print("\n5. DOCUMENT YOUR WORK")
print("   - Keep a development journal")
print("   - Screenshot important results")
print("   - Record process and iterations")
print("   - Prepare portfolio presentation")

if not hf_authenticated:
    print("\n⚠️ RECOMMENDATION:")
    print("   Run Cell 7 again to set up Hugging Face authentication")
    print("   This will give you access to more models and datasets")

print("\n" + "="*60)
print("HELPFUL RESOURCES:")
print("="*60)
print("  • HP AI Studio Docs: https://zdocs.datascience.hp.com/docs/aistudio/")
print("  • Hugging Face: https://huggingface.co/")
print("  • MLflow Documentation: https://mlflow.org/docs/latest/")
print("  • PyTorch Tutorials: https://pytorch.org/tutorials/")
print("  • Gradio Documentation: https://gradio.app/docs/")

print("\n" + "="*60)
print("REMEMBER:")
print("="*60)
print("  • Save your work frequently (Ctrl+S)")
print("  • Document your process in your project journal")
print("  • Test on small datasets before full training")
print("  • Ask for help in office hours if needed")
print("  • Clear GPU memory: torch.cuda.empty_cache()")

print("\n✓ You're ready to begin your AI project!")
print("  Good luck with your creative AI development!")
print("\n" + "="*60)


🎉 SETUP COMPLETE!

Your HP AI Studio environment is configured and ready.
All core dependencies are installed and tested.

✓ GPU: Detected and functional
✓ Hugging Face: Authenticated

NEXT STEPS FOR YOUR AI PROJECT:

1. DEVELOP YOUR MODEL
   - Load datasets using Hugging Face datasets library
   - Fine-tune models or train from scratch
   - Test and evaluate your model performance

2. SAVE YOUR MODEL
   - Use torch.save() for PyTorch models
   - Save tokenizers and configurations
   - Document model architecture and parameters

3. REGISTER FOR DEPLOYMENT
   - Open Register_Model.ipynb
   - Update configuration with your model details
   - Run all cells to register with MLflow
   - Check HP AI Studio Deployments tab

4. CREATE YOUR INTERFACE
   - Use Gradio for interactive UIs
   - Build REST APIs with FastAPI
   - Integrate with existing applications

5. DOCUMENT YOUR WORK
   - Keep a development journal
   - Screenshot important results
   - Record process and iterations
   - Prepar

In [9]:
# PRISM Training Data Generator Setup
import json
import random
import uuid
from datetime import datetime, timedelta
from typing import Dict, List, Optional
from pathlib import Path

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


In [10]:
import json
import random
import uuid
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Optional

# =====================================================
# PERSONAS (6 Types)
# =====================================================
PERSONAS = {
    'perfectionist': {'color_consistency': 0.95, 'typo_rate': 0.01, 'fragment_rate': 0.1},
    'chaotic': {'color_consistency': 0.1, 'typo_rate': 0.25, 'fragment_rate': 0.7},
    'visual': {'color_consistency': 0.7, 'typo_rate': 0.1, 'fragment_rate': 0.3},
    'data_driven': {'color_consistency': 0.85, 'typo_rate': 0.05, 'fragment_rate': 0.2},
    'storyteller': {'color_consistency': 0.6, 'typo_rate': 0.1, 'fragment_rate': 0.2},
    'speedster': {'color_consistency': 0.2, 'typo_rate': 0.3, 'fragment_rate': 0.85}
}

# =====================================================
# COLORS & MAPPINGS
# =====================================================
COLORS = ['RED', 'ORANGE', 'YELLOW', 'GREEN', 'BLUE', 'PURPLE', 'PINK', 'GRAY']

TYPICAL_COLOR_MAP = {
    'pain_point': ['RED', 'ORANGE'],
    'positive': ['GREEN'],
    'neutral': ['YELLOW', 'GRAY'],
    'question': ['BLUE', 'PURPLE'],
    'quote': ['PINK'],
    'idea': ['PURPLE']
}

# =====================================================
# CONTENT LIBRARY
# =====================================================
CONTENT_LIBRARY = {
    'pain_point': [
        "Users can't find the search button",
        "Navigation is too complex",
        "Takes 5+ clicks to complete task",
        "Error message is confusing",
        "Mobile experience is broken",
        "Loading time too slow",
        "Users getting frustrated here",
        "This workflow doesn't make sense",
        "People keep making the same mistake",
        "Accessibility issues with screen readers"
    ],
    'positive': [
        "Users loved this feature!",
        "Really intuitive design",
        "Everyone completed this task easily",
        "Great feedback on the new flow",
        "This is working really well",
        "Users said this was helpful",
        "Clean and simple - people get it",
        "Fast and efficient",
        "Accessibility done right here",
        "Users keep complimenting this"
    ],
    'neutral': [
        "User clicked here first",
        "Average time: 2.5 minutes",
        "Most common path through flow",
        "Users noticed this feature",
        "Standard navigation pattern",
        "Observed behavior: scrolling",
        "Participants mentioned this",
        "Typical user journey",
        "General observation",
        "Users interacted with this element"
    ],
    'question': [
        "Why do users do this?",
        "What's their mental model here?",
        "How often does this happen?",
        "Should we test this with more users?",
        "Is this a real problem or edge case?",
        "What do competitors do?",
        "Can we measure this behavior?",
        "Do users expect something different?",
        "Why did 3 out of 5 users fail here?",
        "What's causing this confusion?"
    ],
    'quote': [
        '"I don\'t understand what to do next"',
        '"This is exactly what I needed"',
        '"Why can\'t I just click here?"',
        '"I expected this to work differently"',
        '"This is really frustrating"',
        '"Love how simple this is"',
        '"I keep making mistakes here"',
        '"Wish I could do this faster"',
        '"Not sure where to find that"',
        '"This saved me so much time"'
    ],
    'idea': [
        "Could we add a shortcut here?",
        "What if we simplified this to 2 steps?",
        "AI suggestion feature?",
        "Add tooltips for first-time users",
        "Keyboard shortcuts would help",
        "Progressive disclosure might work",
        "Could use animation to guide users",
        "Smart defaults based on behavior",
        "Personalization opportunity here",
        "Gamification element?"
    ]
}

# =====================================================
# TEXT MODIFIERS
# =====================================================
FRAGMENTS = ["need to...", "maybe...", "what if...", "...", "???", "TODO:", "hmm", "wait,"]
EMOJI = ["😬", "🤔", "😊", "😤", "🎉", "❗", "💯", "🔥"]
TYPOS = {"the": "teh", "their": "thier", "doesn't": "doesnt", "can't": "cant"}

# =====================================================
# GENERATOR CLASS
# =====================================================
class TrainingDataGenerator:
    def __init__(self, output_dir: str = './synthetic_data'):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        
    def generate_note(self, persona: Dict, content_type: str, use_atypical: bool = False) -> Dict:
        """Generate single sticky note"""
        # Choose content
        content = random.choice(CONTENT_LIBRARY[content_type])
        
        # Apply persona style
        if random.random() < persona['fragment_rate']:
            content = random.choice(FRAGMENTS) + " " + content
        
        if random.random() < persona['typo_rate']:
            for correct, typo in TYPOS.items():
                if correct in content.lower():
                    content = content.replace(correct, typo, 1)
                    break
        
        if random.random() < 0.2:  # 20% chance emoji
            content = content + " " + random.choice(EMOJI)
        
        # Choose color
        if use_atypical or random.random() > persona['color_consistency']:
            color = random.choice(COLORS)  # Random color
        else:
            color = random.choice(TYPICAL_COLOR_MAP[content_type])  # Correct color
        
        # Check if color matches meaning
        is_semantic_match = color in TYPICAL_COLOR_MAP[content_type]
        
        return {
            'id': str(uuid.uuid4())[:8],
            'content': content,
            'color': color,
            'true_type': content_type,
            'color_matches_meaning': is_semantic_match,
            'is_atypical': use_atypical
        }
    
    def generate_board(self, board_num: int, num_notes: int = 50) -> Dict:
        """Generate one FigJam board"""
        # Choose persona
        persona_name = random.choice(list(PERSONAS.keys()))
        persona = PERSONAS[persona_name]
        
        # 15% chance this board uses atypical patterns
        use_atypical = random.random() < 0.15
        
        # Generate notes
        notes = []
        for _ in range(num_notes):
            content_type = random.choice(list(CONTENT_LIBRARY.keys()))
            note = self.generate_note(persona, content_type, use_atypical)
            notes.append(note)
        
        return {
            'board_id': f'board_{board_num:06d}',
            'persona': persona_name,
            'is_atypical': use_atypical,
            'total_notes': len(notes),
            'notes': notes
        }
    
    def generate_dataset(self, num_boards: int = 100, notes_per_board: tuple = (30, 100)):
        """Generate full dataset"""
        print(f"🚀 Generating {num_boards} FigJam boards...")
        print(f"   Output: {self.output_dir}")
        print()
        
        all_boards = []
        total_notes = 0
        
        for i in range(num_boards):
            num_notes = random.randint(*notes_per_board)
            board = self.generate_board(i, num_notes)
            all_boards.append(board)
            total_notes += num_notes
            
            if (i + 1) % 10 == 0:
                print(f"   ✅ Generated {i + 1}/{num_boards} boards...")
        
        # Save to JSON
        output_file = self.output_dir / 'training_data.json'
        with open(output_file, 'w') as f:
            json.dump(all_boards, f, indent=2)
        
        # Save summary
        summary = {
            'total_boards': num_boards,
            'total_notes': total_notes,
            'personas': list(PERSONAS.keys()),
            'content_types': list(CONTENT_LIBRARY.keys()),
            'generated_at': datetime.now().isoformat()
        }
        
        summary_file = self.output_dir / 'summary.json'
        with open(summary_file, 'w') as f:
            json.dump(summary, f, indent=2)
        
        print()
        print("="*60)
        print(f"✅ COMPLETE!")
        print(f"   Boards: {num_boards}")
        print(f"   Notes: {total_notes:,}")
        print(f"   File: {output_file}")
        print("="*60)
        
        return summary

# =====================================================
# QUICK TEST
# =====================================================
if __name__ == "__main__":
    # Generate small test
    gen = TrainingDataGenerator('./synthetic_data/test')
    summary = gen.generate_dataset(num_boards=10, notes_per_board=(20, 50))
    
    print("\n📝 Sample notes:")
    with open('./synthetic_data/test/training_data.json', 'r') as f:
        data = json.load(f)
        for note in data[0]['notes'][:3]:
            print(f"   [{note['color']}] {note['content']}")

🚀 Generating 10 FigJam boards...
   Output: synthetic_data/test

   ✅ Generated 10/10 boards...

✅ COMPLETE!
   Boards: 10
   Notes: 343
   File: synthetic_data/test/training_data.json

📝 Sample notes:
   [PURPLE] ... Progressive disclosure might work
   [YELLOW] Navigation is too complex ❗
   [GRAY] Users interacted with this element 😊


In [12]:
# Generate 100 boards for testing - USE RELATIVE PATH
generator = TrainingDataGenerator('./synthetic_data/test_100')
summary = generator.generate_dataset(
    num_boards=100, 
    notes_per_board=(30, 100)
)

print(f"\n🎯 Ready for AI #2 training!")
print(f"📊 Generated {summary['total_notes']:,} labeled sticky notes")

🚀 Generating 100 FigJam boards...
   Output: synthetic_data/test_100

   ✅ Generated 10/100 boards...
   ✅ Generated 20/100 boards...
   ✅ Generated 30/100 boards...
   ✅ Generated 40/100 boards...
   ✅ Generated 50/100 boards...
   ✅ Generated 60/100 boards...
   ✅ Generated 70/100 boards...
   ✅ Generated 80/100 boards...
   ✅ Generated 90/100 boards...
   ✅ Generated 100/100 boards...

✅ COMPLETE!
   Boards: 100
   Notes: 6,641
   File: synthetic_data/test_100/training_data.json

🎯 Ready for AI #2 training!
📊 Generated 6,641 labeled sticky notes


In [ ]:
# View sample data to see what we generated
import json

with open('./synthetic_data/test_100/training_data.json', 'r') as f:
    data = json.load(f)

print(f"📋 Total boards: {len(data)}")
print(f"📋 First board: {data[0]['board_id']}")
print(f"👤 Persona: {data[0]['persona']}")
print(f"📝 Notes in this board: {data[0]['total_notes']}")
print(f"🎨 Is atypical (breaks color rules)? {data[0]['is_atypical']}")

print(f"\n{'='*70}")
print("📝 SAMPLE NOTES (showing how AI #2 will learn):")
print(f"{'='*70}\n")

for i, note in enumerate(data[0]['notes'][:10], 1):
    match_symbol = "✓" if note['color_matches_meaning'] else "✗"
    print(f"{i}. {match_symbol} [{note['color']:12}] True type: {note['true_type']:12} | {note['content'][:60]}")

print(f"\n💡 The ✗ means color DOESN'T match meaning - this teaches AI #2 to read content, not trust color!")


In [13]:
# Generate FULL dataset - 10,000 boards
print("🚀 Starting FULL dataset generation (10,000 boards)...")
print("⏱️  This will take approximately 5-10 minutes...")
print("="*70)

full_generator = TrainingDataGenerator('./synthetic_data/full_10k')
full_summary = full_generator.generate_dataset(
    num_boards=10000, 
    notes_per_board=(30, 150)
)

print(f"\n{'='*70}")
print("🎉 MASSIVE TRAINING POOL COMPLETE!")
print(f"📊 Total boards: {full_summary['total_boards']:,}")
print(f"📝 Total sticky notes: {full_summary['total_notes']:,}")
print(f"💾 File: ./synthetic_data/full_10k/training_data.json")
print(f"{'='*70}")
print("\n✅ This data is now ready to train AI #2 in your next workbook!")

🚀 Starting FULL dataset generation (10,000 boards)...
⏱️  This will take approximately 5-10 minutes...
🚀 Generating 10000 FigJam boards...
   Output: synthetic_data/full_10k

   ✅ Generated 10/10000 boards...
   ✅ Generated 20/10000 boards...
   ✅ Generated 30/10000 boards...
   ✅ Generated 40/10000 boards...
   ✅ Generated 50/10000 boards...
   ✅ Generated 60/10000 boards...
   ✅ Generated 70/10000 boards...
   ✅ Generated 80/10000 boards...
   ✅ Generated 90/10000 boards...
   ✅ Generated 100/10000 boards...
   ✅ Generated 110/10000 boards...
   ✅ Generated 120/10000 boards...
   ✅ Generated 130/10000 boards...
   ✅ Generated 140/10000 boards...
   ✅ Generated 150/10000 boards...
   ✅ Generated 160/10000 boards...
   ✅ Generated 170/10000 boards...
   ✅ Generated 180/10000 boards...
   ✅ Generated 190/10000 boards...
   ✅ Generated 200/10000 boards...
   ✅ Generated 210/10000 boards...
   ✅ Generated 220/10000 boards...
   ✅ Generated 230/10000 boards...
   ✅ Generated 240/10000 boar

In [14]:
# Save dataset metadata for Brain AI to use later
import json

metadata = {
    'dataset_info': {
        'purpose': 'Training data for PRISM Brain AI #2 (Content Analyzer)',
        'total_boards': 100,  # Will be 10,000 after full generation
        'data_location': './synthetic_data/test_100/training_data.json',
        'full_dataset_location': './synthetic_data/full_10k/training_data.json',
        'created_for': 'AI #2 - Intuitive Content Analysis'
    },
    'data_structure': {
        'each_note_has': [
            'id', 'content', 'color', 'true_type', 
            'color_matches_meaning', 'is_atypical'
        ],
        'content_types': list(CONTENT_LIBRARY.keys()),
        'personas': list(PERSONAS.keys()),
        'colors': COLORS
    },
    'next_steps': [
        '1. Generate full 10K dataset',
        '2. Build Brain AI (data organizer) in new workbook',
        '3. Train Brain AI on this synthetic data',
        '4. Add FigJam API integration for real boards',
        '5. Build final UI in separate workbook'
    ]
}

# Save to file
with open('./prism_dataset_info.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("✅ Dataset info saved to: ./prism_dataset_info.json")
print("\n📋 This file tells your Brain AI where to find the training data!")

✅ Dataset info saved to: ./prism_dataset_info.json

📋 This file tells your Brain AI where to find the training data!
